In [ ]:
import requests
import pandas as pd
import time, sys, os
import argparse
import datetime
import json
import pandas
import numpy

apiroot = 'https://api.coingecko.com/api/v3'
apikey = ''
dex='pancakeswap_new'
volume=1.5e5
lag=6

response_API=requests.get(f"{apiroot}/exchanges/{dex}",  timeout=10).json()["tickers"]
vols = pd.Series(dtype=float)
for i in response_API:
    id_ = i["coin_id"] + "<>" + i["target_coin_id"]
    vols.loc[id_] = i["converted_volume"]["usd"]

vols = vols.sort_values(ascending=False)
vols1 = vols[vols >= volume]
tokens = set()
for pair in vols1.index:
        if "wbnb" in pair or "binance-usd" in pair or "weth" in pair:
                a = pair.split("<>")
                if a[0] in ["wbnb", "binance-usd", "weth"]:
                        tokens.add(a[1]) 
                else:
                        tokens.add(a[0]) 

response_API=requests.get(f"{apiroot}/simple/price",params={"ids": ",".join(tokens),"vs_currencies": "usd","include_24hr_change": "true",},).json()
ret24 = pd.DataFrame()

for k in response_API:
    ret24.loc[k, "24H Return"] = response_API[k]["usd_24h_change"]
# ret24 = ret24[ret24["24H Return"] >= 0]

# retry if timed-out
for coin in ret24.index:
    ret24.loc[coin,"7D Return"]=requests.get(f"{apiroot}/coins/{coin}",  timeout=20).json()["market_data"]["price_change_percentage_7d"]
# retry if timed-out
ret24=ret24.dropna()
for coin in ret24.index:
    response_API=requests.get(f"{apiroot}/coins/{coin}/market_chart",params={"vs_currency": "usd", "days": 1, },timeout=10,).json()["prices"]
    df = pd.DataFrame()
    for i in response_API:
        dt_ = datetime.datetime(1970, 1, 1, 0, 0, 0) + datetime.timedelta(seconds=int(i[0])/1e3)
        df.loc[dt_, "price"] = i[1]
        df = df.sort_index() 

    current = df.index[-1]
    prback = df["price"].asof(current - datetime.timedelta(hours=lag))
    prcurrent = df["price"].iloc[-1]
    intra_ret = (prcurrent - prback) / prback
    ret24.loc[coin, "6H Return"] = intra_ret

df=ret24.copy()   
df = df.sort_values(by="6H Return", ascending=False)
df["24H Return"] = df["24H Return"].apply(lambda x: str(round(x, 2)) + "%")
try:
    df["7D Return"] = df["7D Return"].apply(lambda x: str(round(x, 2)) + "%")
    df["6H Return"] = df["6H Return"].apply(lambda x: str(round(x * 100, 2)) + "%")
except Exception:
    pass
df['DEX']=dex
df=df[['DEX','6H Return','24H Return','7D Return']]


# retry if timed-out
tokens = set()
for pair in df.index:
    tokens.add(pair) 
for token in tokens:
    price=requests.get(f"{apiroot}/simple/price",params={"ids": token,"vs_currencies": "usd","include_24hr_change": "true",},).json()[token]["usd"]
    slprice = price * (1 - 0.05)
    ptprice = price * (1 + 0.05)
    df.loc[token,"Price"]=price
    df.loc[token,"StopLoss"]=slprice
    df.loc[token,"ProfitTarget"]=ptprice

In [6]:
df

,DEX,6H Return,24H Return,7D Return,Price,StopLoss,ProfitTarget
dao-maker,pancakeswap_new,1.38%,2.27%,12.3%,1.630000,1.548500,1.711500
multichain,pancakeswap_new,1.13%,1.92%,0.5%,9.450000,8.977500,9.922500
seedify-fund,pancakeswap_new,0.6%,-2.38%,0.91%,1.330000,1.263500,1.396500
true-usd,pancakeswap_new,0.25%,-0.15%,-0.05%,0.998798,0.948858,1.048738
binance-peg-dogecoin,pancakeswap_new,-0.04%,-1.54%,-2.75%,0.077689,0.073805,0.081573
tether,pancakeswap_new,-0.25%,-0.24%,-0.23%,0.999489,0.949515,1.049463
binance-bitcoin,pancakeswap_new,-0.66%,-0.14%,0.36%,28405.000000,26984.750000,29825.250000
altura,pancakeswap_new,-0.71%,4.28%,23.64%,0.047569,0.045190,0.049947
mobox,pancakeswap_new,-1.15%,-1.29%,-4.1%,0.482720,0.458584,0.506856
marsdao,pancakeswap_new,-1.41%,-3.7%,-1.3%,0.292823,0.278182,0.307464
